# Steps to reproduce

## 01. Conversion

- batch reading
- storing in batches to .pkl files, divided by column sets:
    - sf - simple features
    - tr - train features
    - af - array feautures
    - afexp - array features normalized (i.e. array elements stored as independent rows)

In [1]:
import numpy as np
import pandas as pd
from common import (
    SIMPLE_FEATURE_COLS, ARR_FEATURE_COLS, ALL_TRAIN_COLS
)

class DatasetMetaData:
    def __init__(self, origin_csv_filenames, chunk_filenames_pattern, origin_col_set):
        self.origin_csv_filenames = origin_csv_filenames
        self.chunk_filenames_pattern = chunk_filenames_pattern
        self.origin_col_set = origin_col_set
        self.is_test = 'test_' in chunk_filenames_pattern

**Важно**. В коде ниже в поле `origin_csv_filenames` указаны пути к файлам с тренировочными данными! Имена файлов как в оригинале, файлы лежат в папке `data/`. Если у вас иначе, поправьте.

То же самое с отконвертированными pickle-файлами - они будут сохранены в папке `data/`, см. chunk_filenames_pattern. Если нужно, поправьте под себя

In [ ]:
from io_tools import DatasetConverter

meta_train = DatasetMetaData(
    origin_csv_filenames=['data/train_part_1_v2.csv.gz', 'data/train_part_2_v2.csv.gz'],
    chunk_filenames_pattern='data/train_{label}_{group}_{ind:03d}.pkl',
    origin_col_set=SIMPLE_FEATURE_COLS + ARR_FEATURE_COLS + ALL_TRAIN_COLS
)

DatasetConverter.convert(meta_train)

## 02. Read train

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as sts
import seaborn as sns
from IPython.display import display
from common import *
import warnings

sns.set()
sns.set_style("whitegrid")

color_palette = sns.color_palette('deep') + sns.color_palette('husl', 6) + sns.color_palette('bright') + sns.color_palette('pastel')

%matplotlib inline

In [6]:
%%time
from io_tools import DatasetReader, count_classes

def read_train(cols, rows, foi_expanded=True):
    return DatasetReader.read_dataset(meta_train, cols + train_cols, rows, foi_expanded=foi_expanded)

used_cols = xyz_cols + mom_cols + hit_type_cols + dxyz_cols + exy_cols + edxy_cols + hit_stats_cols + t_cols + ncl_cols + avg_cs_cols
global_feature_importance = None
train, train_foi = read_train(used_cols, 2000000)
display(train.shape, count_classes(train))

(2000000, 63)

(421218, 1578782)

Wall time: 5.39 s


## 03. Feature transformers

Here we declare all transformers. The code a little bit clunky, sorry. Extracted as is from `transformers/err.py`, `tranformers/cosine.py`.

In [7]:
import numpy as np
from common import xy_cols, xyz_cols, ex_cols, ey_cols, exy_cols, dx_cols, dy_cols, edxy_cols, z_cols, t_cols, mom_cols, N_STATIONS
from pipeline import split_classes

err_cols = ['ErrMSE', 'DLL']

nerr_x_cols = ['NErr_X[%i]' % i for i in range(N_STATIONS)]
nerr_y_cols = ['NErr_Y[%i]' % i for i in range(N_STATIONS)]
nerr_xy_cols = nerr_x_cols + nerr_y_cols

err_x_cols = ['Err_X[%i]' % i for i in range(N_STATIONS)]
err_y_cols = ['Err_Y[%i]' % i for i in range(N_STATIONS)]
err_z_cols = ['Err_Z[%i]' % i for i in range(N_STATIONS)]
err_xy_cols = err_x_cols + err_y_cols
err_xyz_cols = err_xy_cols + err_z_cols
ez = np.array([15270., 16470., 17670., 18870.])

def add_mse(data, features):
    dxy = (data.loc[:, xy_cols].values - data.loc[:, exy_cols].values) / data.loc[:, dx_cols + dy_cols].values / 2.
    D = np.nanmean(dxy**2, axis=1)
    
    data.loc[:, err_cols[0]] = D
    features += [err_cols[0]]
    return data

def add_normed_err(data, features):
    dxy = data.loc[:, xy_cols].values - data.loc[:, exy_cols].values
    normed_errors = dxy / np.sqrt(data.loc[:, edxy_cols].values)
    
    for i in range(4):
        data.loc[:, nerr_x_cols[i]] = normed_errors[:, i]
        data.loc[:, nerr_y_cols[i]] = normed_errors[:, i + 4]
    
    features += nerr_xy_cols
    return data

def add_errs(data, features):
    for err_col, e_col, col in zip (err_xy_cols, exy_cols, xy_cols):
        data.loc[:, err_col] = data[e_col].values - data[col].values
        
    for i in range(4):
        data.loc[:, err_z_cols[i]] = ez[i] - data[z_cols[i]].values
    
    features += err_xyz_cols

def create_distr(data):
    dts = [dt.loc[:, err_cols[0]] for dt in split_classes(data)]
    l, r = np.min(data[err_cols[0]]) - 1e-5, np.max(data[err_cols[0]]) + 1e-5
    bins = np.concatenate((
        np.arange(l, 1, .02),
        np.arange(1, 3, .04),
        np.arange(3, 10, .1),
        np.arange(10, 16, .4),
        np.arange(16, 34, 1.),
        np.arange(34, 66, 2),
        np.arange(66, 120, 5.),
        np.linspace(120, r, 3),
    ))
    pdfs = []
    
    for i in range(2):
        pdf, _ = np.histogram(dts[i], bins=bins)
        pdfs.append(pdf)
    
    cdfs = [np.cumsum(pdf) for pdf in pdfs]
    
    return cdfs, pdfs, bins

# выбираем опцию, как считать DLL - либо на основе pdf, либо на основе cdf
def get_dll_pdf(x, pdfs, cdfs, bins):
    def get_probs_pdf(pdf, x):
        indices = np.digitize(x, bins) - 1
        wbin = (bins[indices + 1] - bins[indices]) / (np.max(bins) - np.min(bins))
        prob = pdf[indices] / pdf.sum()
        return prob #* wbin

    probs = [get_probs_pdf(pdf, x) for pdf in pdfs]
    DLL = np.log(probs[1]) - np.log(probs[0])
    return DLL

def get_dll_cdf(x, pdfs, cdfs, bins):
    def get_probs_cdf(cdf, x):
        indices = np.digitize(x, bins) - 1
        wbin = (bins[indices + 1] - bins[0]) / (np.max(bins) - np.min(bins))
        prob = cdf[indices] / cdf[-1]
        return prob * wbin
    probs = [get_probs_cdf(cdf, x) for cdf in cdfs]
    DLL = np.log(probs[1]) - np.log(probs[0])
    return DLL

def add_dll(data, features):
    data[err_cols[1]] = get_dll_pdf(data.loc[:, err_cols[0]], pdfs, cdfs, bins)
    features += err_cols[1:2]
    return data

vm_cols = ['V', 'VT', 'M', 'MT']

def add_velocity(data, features):
    def get_layer_coords(data, cols, i):
        return data[[cols[i], cols[i+4], cols[i+8]]].values
    def get_elayer_coords(data, i):
        exy = data.loc[:, [ex_cols[i], ey_cols[i]]].values
        ez_ = np.tile(ez[i], exy.shape[0]).reshape((-1, 1))
        return np.hstack((exy, ez_))
    def dot(x, y):
        return np.sum(x * y, axis=1, dtype=np.float32)
    def norm(x):
        return np.sqrt(dot(x, x))
    def get_zero_point(data):
        layers = [get_elayer_coords(data, i) for i in range(2)]
        r = layers[1] - layers[0]
        r = r / norm(r)[:, np.newaxis]
        p = get_elayer_coords(data, 0)
        alpha = - p[:, 2] / r[:, 2]
        
        xs = p[:, 0] + alpha * r[:, 0]
        ys = p[:, 1] + alpha * r[:, 1]
        zs = np.tile(0, len(xs))
        return np.vstack((xs, ys, zs)).T
    
    # radius-vector r_i = p_i - p_0: S x N x 3; 
    r = np.array([get_layer_coords(data, xyz_cols, i) for i in range(4)]) - get_zero_point(data)
    
    # time: S x N
    t = data.loc[:, t_cols].values.T
    
    # average velocity avg(r / t): N x 3    
    v_avg = np.nanmean(r / t[:, :, np.newaxis], axis=0)
    # average speed |v|: N
    speed = norm(v_avg)
    # transverse speed |v_xy|: N
    speed_tr = norm(v_avg * np.array([1., 1., 0.]))
    
    # momentum: N
    p = data.loc[:, mom_cols[0]].values
    # transverse momentum: N
    p_tr = data.loc[:, mom_cols[1]].values
    
    # mass: N
    m = p / speed
    # transverse mass: N
    m_tr = p_tr / speed_tr

    results = [speed, speed_tr, m, m_tr]
    for col, res in zip(vm_cols, results):
        data.loc[:, col] = res
    features += vm_cols
    return data

In [8]:
import numpy as np
from common import x_cols, y_cols, z_cols, ex_cols, ey_cols

da_cols = ['DAngle[%d]' % i for i in range(1, 4)]

def add_coses(data, features):
    def get_layer_coords(data, i):
        return data[[x_cols[i], y_cols[i], z_cols[i]]].values
  
    def dot(x, y):
        return np.sum(x * y, axis=1, dtype=np.float32)
    
    def norm(x):
        return np.sqrt(dot(x, x))

    def get_cosine_dist(L1, L2, L1_norm, L2_norm):
        cosines = dot(L1, L2) / L1_norm / L2_norm
        return np.clip(cosines, -1., 1.)
    
    layers = np.array([get_layer_coords(data, i) for i in range(4)])
    layers[1:] -= layers[:3]
    layers[0] = get_zero_point(data)
    
    for i in range(3):
        cur_layer = layers[i]
        next_layer = layers[i+1]
        nan_mask = np.isnan(next_layer[:, 0])
        next_layer[nan_mask, :] = cur_layer[nan_mask, :]
        
        cosines = get_cosine_dist(cur_layer, next_layer, norm(cur_layer), norm(next_layer))
        degrees = to_degrees(cosines)
        cosines[nan_mask] = np.NaN
        degrees[nan_mask] = np.NaN
        data[da_cols[i]] = degrees
        
    features += da_cols
    return data

def to_degrees(cosine):
    return np.arccos(cosine) / np.pi * 180.

def _to_degrees(cosines):
    angles = cosines.copy()
    isn_mask = ~np.isnan(cosines)
    angles[isn_mask] = np.arccos(cosines[isn_mask]) / np.pi * 180.
    return angles

def get_zero_point(data):
    layers = [data[[ex_cols[i], ey_cols[i], z_cols[i]]].values for i in range(2)]
    d = layers[1] - layers[0]
    return d

### 03.1. Build distribution for DLL

In [9]:
# готовим данные для распределения DLL

# либо загружаем уже готовое
# dll_filename = 'data/dll.pkl'
# dll_train = pd.read_pickle(dll_filename)
# display(dll_train.columns)

# либо считаем заново
dll_train, _ = read_train(xy_cols + dx_cols + dy_cols + exy_cols, 10000000)
dll_train = add_mse(train, [])

# опционально пересчитываем MatchedHits и заменяем ими координаты треков в dll_train, чтобы считать распределение на пересчитанных треках
# dll_train = replace_hits(dll_train, [])

# save DLL
# display(dll_train.columns)
# dll_train.to_pickle(dll_filename)

In [10]:
# если распределение для DLL уже есть сохраненное, то загружаем
# cdfs, pdfs, bins = np.load('data/train_cdfs.pkl.npy'), np.load('data/train_pdfs.pkl.npy'), np.load('data/train_bins.pkl.npy')

# либо считаем на основе загруженного dll_train
cdfs, pdfs, bins = create_distr(dll_train)
# np.save('data/train_cdfs.pkl.npy', cdfs)
# np.save('data/train_pdfs.pkl.npy', pdfs)
# np.save('data/train_bins.pkl.npy', bins)

## 04. Declare pipeline

Here we declare pipeline helpers for fit, predict, save model, get stats and so on. Extracted as is from `pipeline.py`.

In [11]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import model_selection as mdsel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import scoring
from common import train_cols

def split_classes(data):
    return [data.loc[data.label == i, :] for i in range(2)]

def count_classes(data):
    cnt_0 = np.count_nonzero(data.label == 0)
    return cnt_0, len(data.index) - cnt_0

def sample(data, nrows):
    return data.iloc[np.random.permutation(len(data.index))[:nrows], :]

def fit(train, n_estimators, transformer_cls):
    labels, weights = get_labels_weights(train.loc[:, train_cols])

    # defined much later
    transformer = transformer_cls().fit(train)
    train_values = transformer.transform(train)
    
    estimator = xgb.XGBClassifier(n_estimators=n_estimators, n_jobs=3)
    estimator.fit(train_values, labels, sample_weight=weights, eval_metric=scoring.rejection90_sklearn)
    return transformer, estimator
    
def predict(fitted_state, test):
    transformer, estimator = fitted_state

    test_value = transformer.transform(test)
    predictions = estimator.predict_proba(test_value)[:, 1]
    return predictions

def score(fitted_state, test):
    labels, weights = get_labels_weights(test.loc[:, train_cols])
    predictions = predict(fitted_state, test)
    return scoring.rejection90(labels, predictions, sample_weight=weights)

def fit_predict_save(train, test, filename, n_estimators, transformer_cls):
    fitted_state = fit(train, n_estimators, transformer_cls)
    predictions = predict(fitted_state, test)
    
    pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
        filename, index_label='id'
    )
    save_model(fitted_state[1], fitted_state[0], filename)
    
def predict_private_save(test, model_fname, filename, transformer_cls):
    fitted_state = (transformer_cls(), xgb.XGBClassifier())
    fitted_state[1].load_model(fname=model_fname)
    
    predictions = predict(fitted_state, test)
    
    pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
        filename, index_label='id'
    )

def fit_save_model(train, filename, n_estimators, transformer_cls):
    transformer, model = fit(train, n_estimators, transformer_cls)
    save_model(model, transformer, filename)

def cv_step(train_subset, test_subset, n_estimators, transformer_cls):
    fit_state = fit(train_subset, n_estimators, transformer_cls)
        
    labels, weights = get_labels_weights(test_subset[train_cols])
    predictions = predict(fit_state, test_subset)

    y_true = labels
    l, r, _ = scoring.get_threshold_details(y_true, predictions, sample_weight=weights)
    threshold = (l + r) / 2
    y_pred = predictions >= threshold

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, predictions)
    scr = scoring.rejection90(y_true, predictions, sample_weight=weights)
    
    scores = [acc, prec, rec, f1, roc_auc, scr, threshold]
    feature_importance = get_xgb_imp(fit_state[1], fit_state[0].features)
    return scores, feature_importance
    
def cross_validate(train, n_estimators, n_splits, n_rows, transformer_cls):
    train = sample(train, n_rows)
    results = [
        cv_step(train.iloc[train_indices, :], train.iloc[test_indices, :], n_estimators, transformer_cls)
        for i, (train_indices, test_indices) in enumerate(mdsel.KFold(n_splits=max(n_splits, 3), shuffle=True).split(train, train.label))
        if i < n_splits
    ]
    scores = [s for s, _ in results]
    feature_importance = sum([fi for _, fi in results]) / n_splits
        
    descr = pd.DataFrame(scores, columns=['acc', 'prec', 'rec', 'f1', 'roc_auc', 'scr', 'th'])
    feature_importance = feature_importance.sort_values(by='score', ascending=False)
    return descr, feature_importance

def get_labels_weights(data):
    return data.label.values, data.weight.values

def save_model(model, transformer, filename):
    model.save_model(to_model_filename(filename))
    with open(to_cols_filename(filename), 'w') as txt_file:
        str_arr = map(str, [transformer.origin_features, transformer.new_features])
        to_write = '\n\n'.join(str_arr)
        txt_file.write(to_write)

def to_model_filename(filename):
    return filename.replace('out/', 'models/').replace('.csv', '.xgb')

def to_cols_filename(filename):
    return filename.replace('out/', 'models/').replace('.csv', '.txt').replace('.xgb', '.txt')

def get_xgb_imp(model, feat_names):
    imp_vals = model.get_booster().get_fscore()
    scores = np.array([float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))])
    scores /= scores.sum()
    
    score = pd.DataFrame(data=scores, index=feat_names, columns=['score'])
    return score


## 05. Declare glue for transformers

Here's the class, where we can set which feature transformers and features at all we gonna use.

In [12]:
from sklearn.base import TransformerMixin

def filter_unimportant_features(features):
    if global_feature_importance is None:
        return features
    fscore = global_feature_importance
    return [col for col in features if col not in fscore.index or fscore.loc[col, 'score'] > 0.00]
    return features

class DataTransformer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
    
    def fit(self, data, y=None):
        return self

    def transform(self, data):
        data = data.copy()
#         features = [] + mom_cols + hit_type_cols + exy_cols + edxy_cols + hit_stats_cols + t_cols + ncl_cols + avg_cs_cols + dxyz_cols + xyz_cols

        # final submission was w/ this set of origin columns
        features = [
            'P', 'PT', 'MatchedHit_TYPE[0]', 'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 
            'Lextra_X[0]', 'Lextra_X[1]', 'Lextra_X[2]', 'Lextra_X[3]',
            'Lextra_Y[0]', 'Lextra_Y[1]', 'Lextra_Y[2]', 'Lextra_Y[3]',
            'Mextra_DX2[0]', 'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]',
            'Mextra_DY2[0]', 'Mextra_DY2[1]', 'Mextra_DY2[3]', 'FOI_hits_N',
            'NShared', 'MatchedHit_T[0]', 'MatchedHit_T[2]',
            'ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]',
            'avg_cs[0]', 'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]',
            'MatchedHit_DX[1]', 'MatchedHit_DX[2]', 'MatchedHit_DX[3]',
            'MatchedHit_DY[0]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[2]'
        ]
        
        features = filter_unimportant_features(features)
        self.origin_features = features.copy()

        add_coses(data, features)
        add_mse(data, features)
        add_normed_err(data, features)
        add_dll(data, features)
        add_errs(data, features)
        add_velocity(data, features)
        
        if not features:
            raise('no features')
    
        features = filter_unimportant_features(features)
#         self.new_features = features[len(self.origin_features):]
        
        # final submission was w/ this set of new features
        self.new_features = [
            'DAngle[1]', 'DAngle[2]', 'DAngle[3]', 'ErrMSE',
            'NErr_X[0]', 'NErr_X[1]', 'NErr_X[2]', 'NErr_X[3]',
            'NErr_Y[0]', 'NErr_Y[1]', 'NErr_Y[2]', 'NErr_Y[3]',
            'DLL', 'Err_X[0]', 'Err_X[1]', 'Err_X[2]', 'Err_X[3]',
            'Err_Y[0]', 'Err_Y[1]', 'Err_Y[3]',
            'Err_Z[0]', 'Err_Z[1]', 'Err_Z[2]', 'Err_Z[3]',
            'V', 'VT', 'M', 'MT'
        ]
        self.features = self.origin_features + self.new_features
        return data[features].values

## 06. Test all is ok

In [13]:
%%time
global_feature_importance = None
df_scores, feature_importance = cross_validate(train, n_estimators=10, n_splits=5, n_rows=1000, transformer_cls=DataTransformer)
display(df_scores.describe())
# display(feature_importance)

,acc,prec,rec,f1,roc_auc,scr,th
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.781000,0.842410,0.889186,0.862179,0.736546,0.462138,0.305616
std,0.006519,0.045320,0.069788,0.007524,0.023514,0.231075,0.070821
min,0.775000,0.773869,0.812500,0.852459,0.710680,0.145398,0.215474
25%,0.775000,0.837349,0.854305,0.859873,0.717491,0.434010,0.268212
50%,0.780000,0.838509,0.882353,0.860000,0.735312,0.442666,0.315249
75%,0.785000,0.865772,0.896774,0.866044,0.752117,0.493119,0.322780
max,0.790000,0.896552,1.000000,0.872521,0.767131,0.795496,0.406364


Wall time: 1.25 s


## 07. Train model

Here we train model and save:

- model to `models/<name>.xgb`
- columns we used to `models/<name>.txt`. That's how we can restore which features were used :)

On my notebook this model was trained for ~1 hour.

In [14]:
%%time
_t = global_feature_importance
# global_feature_importance = None
fit_save_model(train, "models/15_reproduce.xgb", n_estimators=400, transformer_cls=DataTransformer)
global_feature_importance = _t

Wall time: 1min 2s
